## Part 1: Explore the Data

Import the data and use Pandas to learn more about the dataset.

In [1]:
import pandas as pd

df = pd.read_csv('Resources/client_dataset.csv')

df.head()

,first,last,job,phone,email,client_id,order_id,order_date,order_week,order_year,item_id,category,subcategory,unit_price,unit_cost,unit_weight,qty,line_number
0,Donald,Harding,Immunologist,793-904-7725x39308,harding.donald.7185@sullivan.com,58515,8953482,2023-04-28,17,2023,EUD29711-63-6U,decor,wall art,1096.80,762.71,7.50,105,1
1,Tiffany,Myers,Music therapist,201.442.4543x942,myers.t.6537@ferguson-johnson.net,37609,8069089,2023-05-19,20,2023,XDA18116-89-4A,consumables,pens,24.95,15.09,1.49,21,0
2,Shannon,Watson,Immunologist,687.737.9424x8503,swatson8146@payne.net,57113,1902144,2023-01-29,4,2023,ABE59463-05-7E,software,project management,13.52,7.86,1.68,39,6
3,Nathan,Baker,Accounting technician,827-788-8123x012,bakernathan@benson.com,46554,9031802,2023-04-25,17,2023,ZMM00836-65-0C,consumables,pens,36.42,24.85,1.23,29,3
4,Christina,Schwartz,Chiropractor,265-829-3643,christinaschwartz9252@mcconnell.com,92089,1322274,2023-05-28,21,2023,BZX55559-12-3X,consumables,misc,195.10,108.17,46.43,20,1


In [2]:
# View the column names in the data
df.columns


Index(['first', 'last', 'job', 'phone', 'email', 'client_id', 'order_id',
       'order_date', 'order_week', 'order_year', 'item_id', 'category',
       'subcategory', 'unit_price', 'unit_cost', 'unit_weight', 'qty',
       'line_number'],
      dtype='object')

In [3]:
# Use the describe function to gather some basic statistics
df.describe()


,client_id,order_id,order_week,order_year,unit_price,unit_cost,unit_weight,qty,line_number
count,54639.000000,5.463900e+04,54639.000000,54639.000000,54639.000000,54639.000000,54639.000000,5.463900e+04,54639.000000
mean,54837.869416,5.470190e+06,11.359139,2022.993064,136.267207,99.446073,5.004116,5.702646e+02,2.979667
std,25487.438231,2.599807e+06,7.023499,0.082997,183.873135,133.164267,5.326599,1.879552e+04,2.436320
min,10033.000000,1.000886e+06,1.000000,2022.000000,0.010000,0.010000,0.000000,0.000000e+00,0.000000
25%,33593.000000,3.196372e+06,6.000000,2023.000000,20.800000,14.840000,1.450000,3.200000e+01,1.000000
50%,53305.000000,5.496966e+06,11.000000,2023.000000,68.310000,49.890000,3.240000,6.800000e+01,3.000000
75%,78498.000000,7.733869e+06,17.000000,2023.000000,173.160000,125.570000,6.890000,1.700000e+02,5.000000
max,99984.000000,9.998480e+06,52.000000,2023.000000,1396.230000,846.270000,46.430000,3.958244e+06,9.000000


In [4]:
# Use this space to do any additional research
# and familiarize yourself with the data.


In [5]:
# What three item categories had the most entries?
df['category'].value_counts().head(3)

category
consumables    23538
furniture      11915
software        8400
Name: count, dtype: int64

In [6]:
# For the category with the most entries,
# which subcategory had the most entries?
df.subcategory[df.category == "consumables"].value_counts().head(1)

subcategory
bathroom supplies    6424
Name: count, dtype: int64

In [7]:
# Which five clients had the most entries in the data?
top_five_client = df['client_id'].value_counts().head(5)
top_five_client

client_id
33615    220
66037    211
46820    209
38378    207
24741    207
Name: count, dtype: int64

In [8]:
# Store the client ids of those top 5 clients in a list.
top_five_client_ids = top_five_client.index.tolist()
top_five_client_ids

[33615, 66037, 46820, 38378, 24741]

In [9]:
# How many total units (the qty column) did the
# client with the most entries order order?
df.loc[df.client_id == 33615, "qty"].sum()

64313

## Part 2: Transform the Data
Do we know that this client spent the more money than client 66037? If not, how would we find out? Transform the data using the steps below to prepare it for analysis.

In [10]:
# Create a column that calculates the 
# subtotal for each line using the unit_price
# and the qty
df['line_subtotal'] = df['unit_price'] * df['qty']
df[['unit_price', 'qty', 'line_subtotal']].head(3)

,unit_price,qty,line_subtotal
0,1096.80,105,115164.00
1,24.95,21,523.95
2,13.52,39,527.28


In [11]:
# Create a column for shipping price.
# Assume a shipping price of $7 per pound
# for orders over 50 pounds and $10 per
# pound for items 50 pounds or under.
def shipping_cal(weight):
    if weight > 50:
        return weight * 7
    else:
        return weight * 10
df["total_weight"] = df["qty"] * df["unit_weight"]
df["shipping_price"] = df["total_weight"].apply(shipping_cal)
df[["unit_price", "unit_weight", "total_weight", "shipping_price"]].head(3)

,unit_price,unit_weight,total_weight,shipping_price
0,1096.80,7.50,787.50,5512.50
1,24.95,1.49,31.29,312.90
2,13.52,1.68,65.52,458.64


In [12]:
# Create a column for the total price
# using the subtotal and the shipping price
# along with a sales tax of 9.25%

df['line_price'] = round((df['line_subtotal'] + df['shipping_price']) * 1.0925, 2)
df[['line_price', 'line_subtotal', 'shipping_price']].head(3)

,line_price,line_subtotal,shipping_price
0,131839.08,115164.00,5512.50
1,914.26,523.95,312.90
2,1077.12,527.28,458.64


In [13]:
# Create a column for the cost
# of each line using unit cost, qty, and
# shipping price (assume the shipping cost
# is exactly what is charged to the client).

df['line_cost'] = df['unit_cost'] * df['qty'] + df['shipping_cost']
df.head(3)


KeyError: 'shipping_cost'

In [ ]:
# Create a column for the profit of
# each line using line cost and line price
df['line_profit'] = df['line_price'] - df['line_cost']
df.head(3)


## Part 3: Confirm your work
You have email receipts showing that the total prices for 3 orders. Confirm that your calculations match the receipts. Remember, each order has multiple lines.

Order ID 2742071 had a total price of \$152,811.89

Order ID 2173913 had a total price of \$162,388.71

Order ID 6128929 had a total price of \$923,441.25


In [15]:
# Check your work using the totals above
orders_list = [2742071, 2173913, 6128929]

def locate_order_price(order_id):
    orders_df = df.loc[df['order_id'] == order_id, 'line_price']
    return round(orders_df.sum(), 2)

for order_id in orders_list:
    print(f'Order {order_id} total: ${locate_order_price(order_id)}')


Order 2742071 total: $152811.89
Order 2173913 total: $162388.71
Order 6128929 total: $923441.25


## Part 4: Summarize and Analyze
Use the new columns with confirmed values to find the following information.

In [18]:
# How much did each of the top 5 clients by quantity
# spend? Check your work from Part 1 for client ids.
def locate_client_price(client_id, column):
    clients_df = df.loc[df['client_id'] == client_id, column]
    return round(clients_df.sum(), 2)

for client_id in top_five_client:
    print(f'{client_id}: ${locate_client_price(client_id, "line_price")}')

220: $0.0
211: $0.0
209: $0.0
207: $0.0
207: $0.0


In [26]:
# Create a summary DataFrame showing the totals for the
# for the top 5 clients with the following information:
# total units purchased, total shipping price,
# total revenue, and total profit. Sort by total profit.
data_dict = {'client_id': top_five_client}

columns = ['qty', 'shipping_price', 'lime_price', 'lime_cost', 'line_profit']
for column in columns:
    data_dict[column] = [find_client_price(client_id, column) for client_id in top_five_client]
    
sum_df = pd.DataFrame(data_dict).sort_values('line_profits', ascending=False)
sum_df


NameError: name 'client_price' is not defined

In [ ]:
# Format the data and rename the columns
# to names suitable for presentation.
# Currency should be in millions of dollars.



In [ ]:
# Sort the updated data by "Total Profit" form highest to lowest
